In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.mllib.random import RandomRDDs
import statistics as st

In [2]:
# Create SparkSession
spark = SparkSession.builder.appName("asheesh").getOrCreate()

22/03/24 13:18:33 WARN Utils: Your hostname, a-Lenovo-Legion-Y530-15ICH resolves to a loopback address: 127.0.1.1; using 192.168.1.12 instead (on interface wlp7s0)
22/03/24 13:18:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/24 13:18:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/24 13:18:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Transformation
def generate_random_uniform_df(nrows=10, ncols=10,numPartitions=10):
    return RandomRDDs.uniformVectorRDD(spark.sparkContext, nrows,ncols,numPartitions).map(lambda a : a.tolist()).toDF()

In [4]:
# Randomly generated data 
df=generate_random_uniform_df(nrows=100000000,ncols=10,numPartitions=100)
df=df.withColumn("user_vector", F.udf(lambda *z: [i for i in z],ArrayType(DoubleType()))(*df.columns)).select("user_vector")

In [9]:
# Uses of 3rd party libraries 
df=df.withColumn("mean", F.udf(lambda z: st.mean(z),FloatType())(df.user_vector))
df=df.withColumn("std", F.udf(lambda z: st.stdev(z),FloatType())(df.user_vector))

In [10]:
df.show()

+--------------------+----------+----------+
|         user_vector|      mean|       std|
+--------------------+----------+----------+
|[0.87697726541112...| 0.5417995|  0.306426|
|[0.53055591205546...|  0.524963|0.24418919|
|[0.65301240039994...| 0.6427852|0.21293461|
|[0.86995666899085...|0.49199462|0.30135396|
|[0.38864223088077...|0.44524068|0.26834714|
|[0.28606963312762...|0.38847983|0.21260986|
|[0.67647010595842...|0.34423324| 0.2218186|
|[0.27338172584511...|0.39749137| 0.2933082|
|[0.66828749017683...|0.53543127|0.35441568|
|[0.62790165180745...|0.53084034|0.27913564|
|[0.07611015118351...|0.38995948| 0.3176411|
|[0.48345168351790...| 0.5331065|0.27594638|
|[0.11412816515693...|   0.50413| 0.3267447|
|[0.66149339392819...|0.50039935|0.21922049|
|[0.77972261075131...| 0.5958385|0.29511285|
|[0.20912191160759...| 0.5412565|0.31749663|
|[0.79710124484913...| 0.5459609| 0.2381312|
|[0.81542616377014...| 0.5713523|0.35562277|
|[0.54037470017968...| 0.4777771| 0.2410556|
|[0.101173

In [11]:
df.printSchema()

root
 |-- user_vector: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- mean: float (nullable = true)
 |-- std: float (nullable = true)

